In [1]:
%pwd

'd:\\recommandation\\MLproject\\research'

In [2]:
%cd ..

d:\recommandation\MLproject


d:\recommandation\rithvik\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
lib = %pwd

In [4]:
import sys

sys.path.append(lib)

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    objective: str
    n_estimators: int
    learning_rate: float
    max_depth: int
    min_child_weight: int
    subsample: float
    colsample_bytree: float
    reg_lambda: int
    target_column: str
    


In [6]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories


In [15]:
from pathlib import Path
from src.mlProject import logger

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE,
        params_filepath = PARAMS_FILE):

        config_filepath = Path(config_filepath)
        params_filepath = Path(params_filepath)

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBRegressor
        schema =  self.params.TARGET_COLUMN
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            objective = params.objective,
            n_estimators = params.n_estimators,
            learning_rate = params.learning_rate, 
            max_depth = params.max_depth,
            min_child_weight = params.min_child_weight,
            subsample = params.subsample,
            colsample_bytree = params.colsample_bytree,
            reg_lambda = params.reg_lambda,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from src.mlProject import logger
import xgboost as xgb
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        
        xg_reg = xgb.XGBRegressor(objective = self.config.objective,
                                n_estimators = self.config.n_estimators,
                                learning_rate = self.config.learning_rate, 
                                max_depth = self.config.max_depth,
                                min_child_weight = self.config.min_child_weight,
                                subsample = self.config.subsample,
                                colsample_bytree = self.config.colsample_bytree,
                                reg_lambda = self.config.reg_lambda)

        xg_reg.fit(train_x, train_y)
        
        joblib.dump(xg_reg, os.path.join(self.config.root_dir, self.config.model_name))
        return os.path.join(self.config.root_dir, self.config.model_name)



In [17]:
STAGE_NAME = "Model Trainer stage"

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_training = ModelTrainer(config=model_trainer_config)
    model_training.train()
    logger.info(f"Model trained successfully and saved at: {model_training.train()}")
except Exception as e:
    logger.exception(f"Error occurred during {STAGE_NAME} execution: {str(e)}")
    raise e

[2024-05-11 20:11:16,420: INFO: common: yaml file: config\config.yaml loaded successfully]


[2024-05-11 20:11:16,468: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-11 20:11:16,472: INFO: common: created directory at: artifacts]
[2024-05-11 20:11:16,475: INFO: common: created directory at: artifacts/model_trainer]
[2024-05-11 20:28:13,494: INFO: 1990927345: Model trained successfully and saved at: artifacts/model_trainer\model.joblib]
